<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/lettria_ingest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neo4j openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
  Created whee

In [ ]:
import os
import requests
import pandas as pd
from langchain.graphs import Neo4jGraph


letria_api_key = ""
os.environ["OPENAI_API_KEY"] = "sk-"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "tachometer-root-medicine"
os.environ["NEO4J_URI"] = ""

In [ ]:
!wget https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/companies.owl

--2023-12-29 09:23:16--  https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/companies.owl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4165 (4.1K) [text/plain]
Saving to: ‘companies.owl’

companies.owl       100%[===================>]   4.07K  --.-KB/s    in 0s      

2023-12-29 09:23:17 (30.3 MB/s) - ‘companies.owl’ saved [4165/4165]



In [ ]:
ontology_url = 'https://api.app.lettria.com/app/knowledge/import'
with open('./companies.owl', 'rb') as file:
  response = requests.post(
      ontology_url,
      files=[('file', ( "companies.owl", file, "application/rdf+xml"))],
      headers={"Authorization":"LettriaProKey " + letria_api_key}
  )
print(response.json())
knowledge_id = response.json()['knowledgeId']

{'message': 'ok', 'knowledgeId': '658e900f6dc344002af6a9f7'}


In [ ]:
graph = Neo4jGraph()
graph.query("CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE")
graph.query("CALL n10s.graphconfig.init()")

[{'param': 'handleVocabUris', 'value': 'SHORTEN'},
 {'param': 'handleMultival', 'value': 'OVERWRITE'},
 {'param': 'handleRDFTypes', 'value': 'LABELS'},
 {'param': 'keepLangTag', 'value': False},
 {'param': 'keepCustomDataTypes', 'value': False},
 {'param': 'applyNeo4jNaming', 'value': False},
 {'param': 'baseSchemaNamespace', 'value': 'neo4j://graph.schema#'},
 {'param': 'baseSchemaPrefix', 'value': 'n4sch'},
 {'param': 'classLabel', 'value': 'Class'},
 {'param': 'subClassOfRel', 'value': 'SCO'},
 {'param': 'dataTypePropertyLabel', 'value': 'Property'},
 {'param': 'objectPropertyLabel', 'value': 'Relationship'},
 {'param': 'subPropertyOfRel', 'value': 'SPO'},
 {'param': 'domainRel', 'value': 'DOMAIN'},
 {'param': 'rangeRel', 'value': 'RANGE'},
 {'param': 'classNamePropName', 'value': 'name'},
 {'param': 'relNamePropName', 'value': 'name'}]

In [ ]:
news_articles = pd.read_csv("https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv")
news_articles.head()

,title,date,text
0,Chevron: Best Of Breed,2031-04-06T01:36:32.000000000+00:00,JHVEPhoto Like many companies in the O&G secto...
1,FirstEnergy (NYSE:FE) Posts Earnings Results,2030-04-29T06:55:28.000000000+00:00,FirstEnergy (NYSE:FE – Get Rating) posted its ...
2,Dáil almost suspended after Sinn Féin TD put p...,2023-06-15T14:32:11.000000000+00:00,The Dáil was almost suspended on Thursday afte...
3,Epic’s latest tool can animate hyperrealistic ...,2023-06-15T14:00:00.000000000+00:00,"Today, Epic is releasing a new tool designed t..."
4,"EU to Ban Huawei, ZTE from Internal Commission...",2023-06-15T13:50:00.000000000+00:00,The European Commission is planning to ban equ...


In [ ]:
def text2rdf(text_chunks):
  api_route = 'https://api.lettria.com/structuration'

  response = requests.post(
      api_route,
      json={
          "documents": text_chunks,
          "options":{
              "rdf_export": True
          }
      },
      headers={"Authorization":"LettriaProKey " + letria_api_key}
  ).json()

  rdf_string = response['rdf']
  return rdf_string

In [ ]:
graph_serialization = text2rdf(["Tomaz is the CEO of Neo4j"])
"neo4j" in graph_serialization.lower()

False

In [ ]:
graph.query('CALL n10s.rdf.import.inline($payload,"Turtle")', params={'payload': graph_serialization})

[{'terminationStatus': 'OK',
  'triplesLoaded': 196,
  'triplesParsed': 196,
  'namespaces': {'ns2': 'https://doc.lettria.com/ontology/1.0/Data/',
   'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
   'ns1': 'https://doc.lettria.com/ontology/1.0/Metadata/',
   'ns0': 'https://doc.lettria.com/ontology/1.0/',
   'owl': 'http://www.w3.org/2002/07/owl#',
   'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
   'skos': 'http://www.w3.org/2004/02/skos/core#'},
  'extraInfo': '',
  'callParams': {'singleTx': True}}]